<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark

In [2]:
!pip install pyspark
!pip install findspark

In [7]:
!pip install pyspark  findspark wget

In [3]:
# Start session

In [4]:
import findspark
findspark.init()

import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

24/06/20 23:25:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 23:25:22 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [10]:

!curl "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv" >> searchterms.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  227k  100  227k    0     0  2171k      0 --:--:-- --:--:-- --:--:-- 2171k


In [11]:
# Load the csv into a spark dataframe

In [14]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

In [15]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [18]:
# Print the number of rows
num_rows = df.count()
print(f"Number of rows: {num_rows}")

# Print the number of columns
num_columns = len(df.columns)
print(f"Number of columns: {num_columns}")

Number of rows: 10000
Number of columns: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [19]:
df.head(5)

[Row(day=12, month=11, year=2021, searchterm='mobile 6 inch'),
 Row(day=12, month=11, year=2021, searchterm='mobile latest'),
 Row(day=12, month=11, year=2021, searchterm='tablet wifi'),
 Row(day=12, month=11, year=2021, searchterm='laptop 14 inch'),
 Row(day=12, month=11, year=2021, searchterm='mobile 5g')]

In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [20]:
df.printSchema()

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [22]:
df.groupby("searchterm").count().filter("searchterm='gaming laptop'").show()

[Stage 27:===================================>                    (48 + 9) / 75]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|gaming laptop|  499|
+-------------+-----+



In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [23]:
df.groupby("searchterm").count().sort("count", ascending=False).show(5)

[Stage 29:==========================================>          (160 + 11) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [24]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2024-06-21 00:00:18--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2024-06-21 00:00:18 (10.3 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [25]:
!tar -xvzf model.tar.gz

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
sales_prediction.model/data/_SUCCESS
sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
sales_prediction.model/data/._SUCCESS.crc


In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [26]:
from pyspark.ml.regression import LinearRegressionModel

model=LinearRegressionModel.load('sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [28]:
from pyspark.ml.feature import VectorAssembler

def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data=[[year,0]]
    columns=["year","sales"]
    _ = spark.createDataFrame(data,columns)
    __ = assembler.transform(_).select('features','sales')
    predictions = model.transform(__)
    predictions.select('prediction').show()
    
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+

